# Imports

In [21]:
!python ../src/cleaning.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
from pathlib import Path

c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\src\cleaning.py:69: DtypeWarning: Columns (0: Unit4UUID, 1: Oral Reading UIN, 2: Form Code - Unit 4, 3: U4 Attempt, 4: Move On When Reading Requirement, 5: RC1 C4 Cluster Code, 6: RC1 C5 Cluster Code, 7: RC2 C4 Cluster Code, 8: RC3 C4 Cluster Code, 9: RC3 C5 Cluster Code, 10: ORF UIN 1, 11: ORF Status 1, 12: ORF Score 1, 13: ORF UIN 3, 14: ORF Status 3, 15: ORF Score 3, 16: Read Aloud Test Content, 17: Simplified Directions, 18: IEP Designated Magnification, 19: Proctor Name, 20: Proctor Certified, 21: Proctor Selected Magnification) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter="\t")
Traceback (most recent call last):
  File "c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\.venv\Lib\site-packages\pandas\core\indexes\base.py", line 3641, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~

# Paths

In [19]:
src_path = Path.cwd().parent / "src"
sys.path.insert(0, str(src_path))
from config import DATA_PROCESSED

FILE = DATA_PROCESSED / "analysis_file.csv"

# Load Data

In [20]:
df = pd.read_csv(FILE)

print(df.shape)
df.head()

(531, 23)


,student_id,school,intervention,school_name,state_student_id,Gender,Grade,RaceEthnicity,SPED,FRL,...,Gifted,Migrant,CTE,DatePulled,subject_x,pretest_score,subject_y,bm1_score,ly_math_AASA_score,BM1_gain_score
0,431649,CHS,In-Person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,440605,CHS,In-Person,Crismon High School,18646683.0,Male,9.0,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3676.0,Algebra_I,3669.0,NaN,NaN
2,434131,CHS,In-Person,Crismon High School,62425336.0,Female,9.0,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3674.0,Algebra_I,3678.0,NaN,NaN
3,435873,CHS,In-Person,Crismon High School,49872954.0,Female,9.0,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3662.0,Algebra_I,3670.0,NaN,NaN
4,419323,CHS,In-Person,Crismon High School,12196385.0,Female,9.0,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3657.0,Algebra_I,3651.0,NaN,NaN
